In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/FL_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9794577700,FL,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,1345247301,FL,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14
2,1442247205,FL,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14
3,2030068400,FL,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14
4,2964268209,FL,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9794577700,FL,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,1345247301,FL,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14
2,1442247205,FL,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14
3,2030068400,FL,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14
4,2964268209,FL,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1805,4507747110,FL,"1016 THOMAS DR, P C BEACH, FL",30.175531,-85.760657,NaN
4581,2637917101,FL,"660 US Highway One, NORTH PALM BEACH, FL",43.516228,-70.429591,NaN
5285,1797788503,FL,"19370 COLLINS AVE APT 905, SUNNY ISLES BEACH, FL",25.954947,-80.122033,NaN
6154,7500287104,FL,"3311 HIGHWAY 29, CANTONMENT, FL",36.639637,-79.385116,NaN
6826,2398897110,FL,"300 Industrial Park Dr, Perry, FL",32.463852,-83.770625,NaN
...,...,...,...,...,...,...
55585,9636567310,FL,"7921 VENTURE CENTER WAY APT 1207, BOYNTON BEAC...",26.524504,-80.170804,NaN
56913,7790367406,FL,"7932 Dilido Boulevard, MIRAMAR, FL",25.977540,-80.245599,NaN
56951,3620167110,FL,"2741 Hartley Lane, LITHIA, FL",27.882383,-82.076611,NaN
57456,4732607208,FL,"114 S Orange Ave, Orlando, FL",28.540996,-81.379197,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('12')] #FL
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/72 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

72


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_6419/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1805,4507747110,FL,"1016 THOMAS DR, P C BEACH, FL",30.175531,-85.760657,[120050026043000]
4581,2637917101,FL,"660 US Highway One, NORTH PALM BEACH, FL",43.516228,-70.429591,[]
5285,1797788503,FL,"19370 COLLINS AVE APT 905, SUNNY ISLES BEACH, FL",25.954947,-80.122033,[120860001151000]
6154,7500287104,FL,"3311 HIGHWAY 29, CANTONMENT, FL",36.639637,-79.385116,[]
6826,2398897110,FL,"300 Industrial Park Dr, Perry, FL",32.463852,-83.770625,[]
...,...,...,...,...,...,...
55585,9636567310,FL,"7921 VENTURE CENTER WAY APT 1207, BOYNTON BEAC...",26.524504,-80.170804,[120990059501002]
56913,7790367406,FL,"7932 Dilido Boulevard, MIRAMAR, FL",25.977540,-80.245599,[120111104021007]
56951,3620167110,FL,"2741 Hartley Lane, LITHIA, FL",27.882383,-82.076611,[120570131002017]
57456,4732607208,FL,"114 S Orange Ave, Orlando, FL",28.540996,-81.379197,[120950189021035]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_6419/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1805,4507747110,FL,"1016 THOMAS DR, P C BEACH, FL",30.175531,-85.760657,120050026043000
4581,2637917101,FL,"660 US Highway One, NORTH PALM BEACH, FL",43.516228,-70.429591,NaN
5285,1797788503,FL,"19370 COLLINS AVE APT 905, SUNNY ISLES BEACH, FL",25.954947,-80.122033,120860001151000
6154,7500287104,FL,"3311 HIGHWAY 29, CANTONMENT, FL",36.639637,-79.385116,NaN
6826,2398897110,FL,"300 Industrial Park Dr, Perry, FL",32.463852,-83.770625,NaN
...,...,...,...,...,...,...
55585,9636567310,FL,"7921 VENTURE CENTER WAY APT 1207, BOYNTON BEAC...",26.524504,-80.170804,120990059501002
56913,7790367406,FL,"7932 Dilido Boulevard, MIRAMAR, FL",25.977540,-80.245599,120111104021007
56951,3620167110,FL,"2741 Hartley Lane, LITHIA, FL",27.882383,-82.076611,120570131002017
57456,4732607208,FL,"114 S Orange Ave, Orlando, FL",28.540996,-81.379197,120950189021035


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
4581,2637917101,FL,"660 US Highway One, NORTH PALM BEACH, FL",43.516228,-70.429591,NaN
6154,7500287104,FL,"3311 HIGHWAY 29, CANTONMENT, FL",36.639637,-79.385116,NaN
6826,2398897110,FL,"300 Industrial Park Dr, Perry, FL",32.463852,-83.770625,NaN
9005,4503367106,FL,"VERNIS & BOWLING OF MIAMI PA, MIAMI, FL",41.203322,-77.194525,NaN
9098,7528697002,FL,"2001 N US HWY 27, HOLLYWOOD, FL",32.850258,-84.845482,NaN
24025,3778147104,FL,"1645 Florida Line Road, Quincy, FL",30.701605,-84.626124,NaN
28101,4539788907,FL,"31818 US Hwy 19, Palm Beach, FL",30.997660,-84.046845,NaN
33331,9595917006,FL,"21220 US 19 North, CLEARWATER, FL",30.923185,-83.988545,NaN
35506,2514597308,FL,"1006 NORTH US HWY 27, FORT LAUDERDALE, FL",32.850258,-84.845482,NaN
36766,8854147208,FL,"10 Parachute KY Lot 51, HOMESTEAD, FL",37.839333,-84.270018,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
1805,4507747110,120050026043000
4581,2637917101,NaN
5285,1797788503,120860001151000
6154,7500287104,NaN
6826,2398897110,NaN
...,...,...
55585,9636567310,120990059501002
56913,7790367406,120111104021007
56951,3620167110,120570131002017
57456,4732607208,120950189021035


Read in base data

In [15]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/FL_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,204217,9794577700,FL,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,204218,1345247301,FL,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14
2,204219,1442247205,FL,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14
3,204220,2030068400,FL,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14
4,204221,2964268209,FL,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14


Merge with MainFrame

In [16]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,204217,9794577700,FL,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14,NaN
1,204218,1345247301,FL,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14,NaN
2,204219,1442247205,FL,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14,NaN
3,204220,2030068400,FL,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14,NaN
4,204221,2964268209,FL,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14,NaN
...,...,...,...,...,...,...,...,...
59998,264215,9950428308,FL,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,1.201109e+14,NaN
59999,264216,9963928609,FL,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,1.209901e+14,NaN
60000,264217,9977848305,FL,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,1.210103e+14,NaN
60001,264218,9988227001,FL,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,1.205700e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [17]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,204217,9794577700,FL,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14,NaN,121090208023015.0
1,204218,1345247301,FL,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14,NaN,120950153001006.0
2,204219,1442247205,FL,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14,NaN,120860078082020.0
3,204220,2030068400,FL,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14,NaN,121050143012025.0
4,204221,2964268209,FL,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14,NaN,120570119082004.0


Drop Extra FIPS Columns

In [18]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9794577700,FL,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,121090208023015.0
1,1345247301,FL,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,120950153001006.0
2,1442247205,FL,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,120860078082020.0
3,2030068400,FL,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,121050143012025.0
4,2964268209,FL,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,120570119082004.0
...,...,...,...,...,...,...
59998,9950428308,FL,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,120110906012023.0
59999,9963928609,FL,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,120990069101014.0
60000,9977848305,FL,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,121010310082016.0
60001,9988227001,FL,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,120570026001008.0


In [19]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
4581,2637917101,FL,"660 US Highway One, NORTH PALM BEACH, FL",43.516228,-70.429591,NaN
6154,7500287104,FL,"3311 HIGHWAY 29, CANTONMENT, FL",36.639637,-79.385116,NaN
6826,2398897110,FL,"300 Industrial Park Dr, Perry, FL",32.463852,-83.770625,NaN
9005,4503367106,FL,"VERNIS & BOWLING OF MIAMI PA, MIAMI, FL",41.203322,-77.194525,NaN
9098,7528697002,FL,"2001 N US HWY 27, HOLLYWOOD, FL",32.850258,-84.845482,NaN
24025,3778147104,FL,"1645 Florida Line Road, Quincy, FL",30.701605,-84.626124,NaN
28101,4539788907,FL,"31818 US Hwy 19, Palm Beach, FL",30.997660,-84.046845,NaN
33331,9595917006,FL,"21220 US 19 North, CLEARWATER, FL",30.923185,-83.988545,NaN
35506,2514597308,FL,"1006 NORTH US HWY 27, FORT LAUDERDALE, FL",32.850258,-84.845482,NaN
36766,8854147208,FL,"10 Parachute KY Lot 51, HOMESTEAD, FL",37.839333,-84.270018,NaN


Convert to File

In [20]:
fips_FL = fips_merge_drop

In [21]:
fips_FL.to_csv("../../../data/state_data/geo/geo_fips/23/FL_fips_scraped.csv") 